In [ ]:
# usual imports
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Census 1

In [ ]:
# As of Jan 2017, this census file is online here: 
data = 'http://www2.census.gov/programs-surveys/popest/datasets/2010-2015/national/asrh/nc-est2015-agesex-res.csv'

# A copy can be accessed here in case census.gov moves the file:
# data = 'http://inferentialthinking.com/notebooks/nc-est2015-agesex-res.csv'
# or via a local copy
# data = 'nc-est2015-agesex-res.csv'


full_census_table = Table.read_table(data)

partial = full_census_table.select(['SEX', 'AGE', 4, 9])
us_pop = partial.relabeled(2, '2010').relabeled(3, '2015')
ratio = (us_pop.column(3) / us_pop.column(2))

census = us_pop.with_columns(  # add columns with computed info
        'Change', us_pop.column(3) - us_pop.column(2), 
        'Total Growth', ratio - 1,
        'Annual Growth', ratio ** (1/5) - 1)
census.set_format([2, 3, 4], NumberFormatter)  # show column as percentage
census.set_format([5, 6], PercentFormatter)

In [ ]:
# SEX == 1 is female, == 2 is male, 0 is the combined total 
# select out "both" entries
by_age = census.where('SEX', 0).drop('SEX').where('AGE', are.below(999))
by_age

In [ ]:
# plot 2010 and 2015 population vs age 
# (Would be easier to understand if the column names were used)
by_age.select(0, 1, 2).plot(0)

In [ ]:
# plot change by age; does the structure make sense?
by_age.select("AGE", "Change").plot(0)

In [ ]:
# which ages have the most change?
# 'Change' is the number difference from 2010 to 2015 at that age
# 'Total Growth' and 'Annual Growth' are 2010 to 2015 percentages at that age
by_age.sort("Change", descending=True)

In [ ]:
# When were those people born?
2010-68 

In [ ]:
2015-68 

In [ ]:
# What do you think is the cause of that big increase?

In [ ]:
# look at the top end (100 bin includes 101, 102, etc)
by_age.where('AGE', are.above(89)).show()

In [ ]:
# Can you show whether people are living longer or not? 
# How do you account for birth rate and immigration differences?

In [ ]:
# As a hint, let's add a column that's the difference from year to year
# i.e. 2010 age 1 - 2010 age 0 is 6558 more people
with_delta2010 = by_age.with_column('2010 Diff', by_age.to_df()['2010'].diff())
with_delta2010.where('AGE', are.above(89)).show()
# Can you make this a fraction?  Do the same for 2015?

In [ ]:
# Repeat to make a fraction and display as a percent
with_delta2010 = by_age.with_column('2010 Diff', by_age.to_df()['2010'].diff() / by_age.to_df()['2010'])
with_delta2010.set_format('2010 Diff', PercentFormatter)
with_delta2010.where('AGE', are.above(89)).show()

In [ ]:
# and add the 2015 column
with_delta = with_delta2010.with_column('2015 Diff', with_delta2010.to_df()['2015'].diff() / with_delta2010.to_df()['2015'])
with_delta.set_format('2015 Diff', PercentFormatter)
with_delta.where('AGE', are.above(89)).show()

In [ ]:
# plot as a bar plot for easier scanning and interpretation
for_bar_plot = with_delta.drop("Change", "2010", "2015", "Total Growth", "Annual Growth")
for_bar_plot.barh('AGE')

In [ ]:
# Look at the changes by age between 60 and 70. See anything striking?